In [ ]:
from datasets import load_dataset
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,balanced_accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Subset
from transformers import EarlyStoppingCallback
from sklearn.model_selection import StratifiedKFold 
import numpy as np
from datasets import Dataset, DatasetDict, ClassLabel


In [1]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Set the environment variable for CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Load the dataset from the CSV file
dataset = load_dataset(
    'csv',
    data_files='datasethumanattribution.csv',
    delimiter=',',
    column_names=[
        "folder","filename","claim","label","url","premise","category","gemini_label","gemini_explanation","gpt4_label","gpt4_rationale","split"
    ],
    skiprows=1
)

# Keep only the required columns
columns_to_keep = ["claim", "premise", "label"]
all_columns = dataset['train'].column_names
columns_to_drop = [col for col in all_columns if col not in columns_to_keep]

for split in dataset.keys():
    dataset[split] = dataset[split].remove_columns(columns_to_drop)

# Define label mappings
label2id = {
    "contradiction": 0,
    "entailment": 1,
    "neutral": 2
}

id2label = {v: k for k, v in label2id.items()}

label_mapping = {
    'SUPPORTED': 'entailment',
    'REFUTED': 'contradiction',
    'NOT ENOUGH INFORMATION': 'neutral'
}

def map_and_encode_labels(example):
    # Map original dataset labels to new labels ('entailment', 'contradiction', 'neutral')
    mapped_label = label_mapping.get(example['label'], 'neutral')
    # Encode mapped labels using label2id
    example['label'] = label2id[mapped_label]
    return example

for split in dataset.keys():
    dataset[split] = dataset[split].map(map_and_encode_labels)

# Load pre-trained tokenizer and model
model_name = "pritamdeka/PubMedBERT-MNLI-MedNLI"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.eval()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize the entire dataset
class MediClaimDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer_name='pritamdeka/PubMedBERT-MNLI-MedNLI'):
        self.dataset = dataset
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        
        # Extracting claim and premise texts
        claim = str(item['claim']) if item['claim'] is not None else ""
        premise = str(item['premise']) if item['premise'] is not None else ""
        
        # Tokenize the texts
        inputs = self.tokenizer(
            premise, claim,
            return_tensors="pt",
            padding='max_length',
            truncation=True,
            max_length=512,
            add_special_tokens=True
        )
        
        # Remove batch dimension and add to item
        item['input_ids'] = inputs['input_ids'].squeeze(0)
        item['attention_mask'] = inputs['attention_mask'].squeeze(0)
        
        if 'label' in item:
            item['labels'] = torch.tensor(item['label'], dtype=torch.long)
        
        return item

# Convert entire dataset to MediClaimDataset
full_dataset = MediClaimDataset(dataset['train'], tokenizer_name=model_name)

# Create DataLoader for the full dataset
full_loader = torch.utils.data.DataLoader(full_dataset, batch_size=8, collate_fn=lambda x: x)

# Perform inference
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in full_loader:
        # Ensure no None values in batch
        batch = [b for b in batch if b is not None]
        input_ids = torch.stack([b['input_ids'] for b in batch]).to(device)
        attention_mask = torch.stack([b['attention_mask'] for b in batch]).to(device)
        labels = torch.stack([b['labels'] for b in batch]).to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
balanced_accuracy = balanced_accuracy_score(all_labels, all_predictions)

metrics = {
    'accuracy': accuracy,
    'f1': f1,
    'precision': precision,
    'recall': recall,
    'balanced_accuracy': balanced_accuracy
}

# Map predicted labels back to their original labels
predicted_labels = [id2label[label] for label in all_predictions]

# Print the predictions and metrics
print("Predicted labels:", predicted_labels)
print("Metrics:", metrics)


/home/elson/factcheck/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-6274d2e8b2aaf27b
Reusing dataset csv (/home/elson/.cache/huggingface/datasets/csv/default-6274d2e8b2aaf27b/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
100%|██████████| 1/1 [00:00<00:00, 507.42it/s]
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-6274d2e8b2aaf27b/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-11cf639ca5c8f9ee.arrow


Predicted labels: ['neutral', 'entailment', 'entailment', 'contradiction', 'entailment', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'neutral', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'contradiction', 'contradiction', 'contradiction', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'contradiction', 'entailment', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'contradiction', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'entailment', 'contradiction', 'entailment', 'entailment', 'entailment', 'entailment', 'entailment', 'contradiction', 'contradiction', 'entail

In [ ]:
from datasets import load_dataset

# Load the dataset from the CSV file
dataset = load_dataset(
    'csv',
    data_files='datasethumanattribution.csv',
    delimiter=',',
    column_names=[
        "folder","filename","claim","label","url","premise","category","gemini_label","gemini_explanation","gpt4_label","gpt4_rationale","split"
    ],
    skiprows=1
)


train_dataset = dataset['train'].filter(lambda example: example['split'] == 'train')
validation_dataset = dataset['train'].filter(lambda example: example['split'] == 'validation')
test_dataset = dataset['train'].filter(lambda example: example['split'] == 'test')
dataset = DatasetDict({
    'train': train_dataset,
    'val': validation_dataset,
    'test': test_dataset
})

In [ ]:
dataset

In [ ]:
columns_to_keep = ["claim", "premise", "label","category"]
all_columns = dataset["train"].column_names

columns_to_drop = [col for col in all_columns if col not in columns_to_keep]
for split in dataset.keys():
    dataset[split] = dataset[split].remove_columns(columns_to_drop)

In [ ]:
from datasets import load_dataset, DatasetDict

label2id = {
    "contradiction": 0,
    "entailment": 1,
    "neutral": 2
}

id2label = {v: k for k, v in label2id.items()}

label_mapping = {
    'SUPPORTED': 'entailment',
    'REFUTED': 'contradiction',
    'NOT ENOUGH INFORMATION': 'neutral'
}

def map_and_encode_labels(example):
    # Map original dataset labels to new labels ('entailment', 'contradiction', 'neutral')
    mapped_label = label_mapping[example['label']]
    # Encode mapped labels using label2id
    example['label'] = label2id[mapped_label]
    return example

for split in dataset.keys():
    dataset[split] = dataset[split].map(map_and_encode_labels)


# Show the label encoding mapping
print("Label Encoding Mapping:", label2id)

In [ ]:
labels = np.array(dataset['train']['label'])

In [ ]:
from transformers import AutoTokenizer
import torch.utils.data

class MediClaimDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer_name='pritamdeka/PubMedBERT-MNLI-MedNLI'):
        self.dataset = dataset
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        idx = int(idx)  # Ensure idx is an integer
        item = self.dataset[idx]  # Access the dataset item at idx
        
        # Extracting claim and evidence texts

        claim = str(item['claim'])
        premise = str(item['premise'])
        
        # Tokenize the texts
        inputs = self.tokenizer(
            premise, claim,
            return_tensors="pt",  # Ensure PyTorch tensors are returned
            padding='max_length',  # Apply padding to the maximum length
            truncation='longest_first',  # Truncate to the maximum length if necessary
            max_length=512,  # Specify the maximum length
            add_special_tokens=True  # Add special tokens like [CLS], [SEP]
        )
        
        item['input_ids'] = inputs['input_ids'].squeeze()  # Remove batch dimension
        item['attention_mask']= inputs['attention_mask'].squeeze() # Remove batch dimension
        
        if 'label' in item:
            item['labels'] = torch.tensor(item['label'], dtype=torch.long)
        
        return item



In [ ]:
import torch
print(torch.cuda.device_count())
print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

In [ ]:
model_name = "pritamdeka/PubMedBERT-MNLI-MedNLI"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                 num_labels=3, ignore_mismatched_sizes=True)
device = "cuda:0"
model.to(device)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average="weighted")  # Specify average method
    recall = recall_score(labels, preds, average="weighted")  # Specify average method

    bal_accuracy = balanced_accuracy_score(labels,preds)

    return {"accuracy": acc, "balanced_accuracy":bal_accuracy, "precision": prec, "recall": recall, "f1": f1}

In [ ]:
dataset['train']

In [ ]:
import gc

torch.cuda.set_device(0)

# Clearing the cache
torch.cuda.empty_cache()
gc.collect()
# Checking GPU memory, making sure to reset peak memory stats
torch.cuda.reset_peak_memory_stats()


In [ ]:
current_device = torch.cuda.current_device()
print(f"Current CUDA device: GPU {current_device}")

In [ ]:
train_data = dataset['train']
eval_data = dataset['val']
model = model.to('cuda:0')

In [ ]:
tdata = MediClaimDataset(train_data)
vdata = MediClaimDataset(eval_data)
test_data = MediClaimDataset(dataset['test'])

In [ ]:
tdata.__getitem__(0)

In [ ]:
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments,DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)


training_args = TrainingArguments(
    output_dir=f'/home/elson/1.5.1_pubmedbert/',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    dataloader_pin_memory=True,
    dataloader_num_workers=4,
    fp16=True,
    warmup_ratio=0.01,
    weight_decay=0.1,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=tdata,
    eval_dataset=vdata,
    #tokenizer=tokenizer,
    #data_collator = data_collator,
    compute_metrics=compute_metrics
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)]
)

# Training and Evaluation
trainer.train()
eval_result = trainer.evaluate(vdata)

# Save the best model and tokenizer
model.save_pretrained(f'/home/elson/1.5.1_pubmedbert/best_model')
tokenizer.save_pretrained(f'/home/elson/1.5.1_pubmedbert/best_model')


In [ ]:
#model_path = "/home/elson/1.5.1_pubmedbert/best_model/"
#model = AutoModelForSequenceClassification.from_pretrained(model_path).to('cuda:0')

# Evaluate on the test set
test_results = trainer.predict(test_data)

In [ ]:
print(test_results)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
probabilities = torch.softmax(torch.tensor(test_results.predictions).to(torch.float32), dim=-1)
predictions = np.argmax(probabilities.numpy(), axis=1)
true_labels = test_results.label_ids
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.ylabel("Actual Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
import pandas as pd
data_to_save = []
for idx in range(len(test_data)):
    item = dataset['test'][idx]
    actual_label = item['label']
    predicted_label = predictions[idx]
    claim = item['claim'] 
    premise = item['premise'] 
    category = item['category']
    
    # Append the information as a dictionary to the list
    data_to_save.append({
        'Claim': claim,
        'Premise': premise,
        'Actual Label': actual_label,
        'Predicted Label': predicted_label,
        'Category' : category
    })

df = pd.DataFrame(data_to_save)

# Save the DataFrame to a CSV file
df.to_csv('/home/elson/results/1.5.1_results.csv', index=False)

In [ ]:
# Calculate correctly classified instances
correctly_classified = df[df['Actual Label'] == df['Predicted Label']]

# Calculate misclassified instances
misclassified = df[df['Actual Label'] != df['Predicted Label']]

# Count the number of correctly classified and misclassified by category
correct_classification_counts = correctly_classified['Category'].value_counts()
misclassification_counts = misclassified['Category'].value_counts()

In [ ]:
correct_classification_counts

In [ ]:
misclassification_counts